In [1]:
import sys
!{sys.executable} -m pip install qdrant-client

  Using cached qdrant_client-1.13.3-py3-none-any.whl.metadata (10 kB)
  Using cached grpcio-1.71.0-cp313-cp313-macosx_10_14_universal2.whl.metadata (3.8 kB)
  Using cached grpcio_tools-1.71.0-cp313-cp313-macosx_10_14_universal2.whl.metadata (5.3 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached h2-4.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
Using cached qdrant_client-1.13.3-py3-none-any.whl (306 kB)
Using cached grpcio-1.71.0-cp313-cp313-macosx_10_14_universal2.whl (11.3 MB)
Using cached grpcio_tools-1.71.0-cp313-cp313-macosx_10_14_universal2.whl (6.0 MB)
Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
Using cached h2-4.2.0-py3-none-any.whl (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00
Using cached hpack-4.1.0-py3-none-any.whl (34 kB)
Using cached hyperframe-6.1.0-py3-none-any.w

In [23]:
import os
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv(os.path.join(os.getcwd(),'input/Suchanfragen.csv'))
df.head()

,Häufigste Suchanfragen,Klicks,Impressionen,CTR,Position
0,checkdomain dns einstellungen,5,51,9.8%,3.08
1,domainfactory dns einstellungen,3,97,3.09%,6.71
2,checkdomain dns einträge,3,66,4.55%,3.06
3,hosteurope nameserver ändern,2,97,2.06%,8.88
4,domainfactory dns settings,2,17,11.76%,6.00


In [24]:
df_qrs = df[['Häufigste Suchanfragen']][:10]
df_qrs = df_qrs.rename(columns={'Häufigste Suchanfragen': 'queries'})
df_qrs['queries'] = df_qrs['queries'].astype(str)
df_qrs.head()

,queries
0,checkdomain dns einstellungen
1,domainfactory dns einstellungen
2,checkdomain dns einträge
3,hosteurope nameserver ändern
4,domainfactory dns settings


In [25]:
import ollama

model = 'avr/sfr-embedding-mistral'

def generate_embeddings(text, model=model):
    return ollama.embeddings(model=model, prompt=text)["embedding"]

df_qrs['embeddings'] = df_qrs["queries"].apply(lambda x : generate_embeddings (x, model = model))

In [47]:
df_qrs.head(10)

,queries,embeddings
0,checkdomain dns einstellungen,"[-2.2756826877593994, 7.907756328582764, 0.105..."
1,domainfactory dns einstellungen,"[0.5061586499214172, 8.097482681274414, 0.3920..."
2,checkdomain dns einträge,"[-1.746508002281189, 7.005468368530273, -0.260..."
3,hosteurope nameserver ändern,"[0.2727486789226532, 6.117392063140869, -1.820..."
4,domainfactory dns settings,"[1.6691151857376099, 6.368677139282227, -1.472..."
5,strato ttl einstellen,"[-1.3807024955749512, 7.738464832305908, -1.34..."
6,checkdomain dns,"[-0.3910132348537445, -4.43722677230835, 7.125..."
7,domainfactory dns,"[0.6811183094978333, 0.11925356835126877, 3.53..."
8,rider proxy,"[1.8679367303848267, -0.09533309191465378, -0...."
9,google analytics einrichten,"[0.2725484371185303, 9.54599666595459, 0.12453..."


In [48]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams

client = QdrantClient(":memory:")
# client = QdrantClient(url="http://localhost:6333")

# client.create_collection(
#     collection_name="test_collection",
#     vectors_config=VectorParams(size=4, distance=Distance.DOT),
# )

vector_size = len(df_qrs["embeddings"][0])

# Check if the collection already exists
# and create it if it doesn't
if not client.collection_exists("test_collection"):
   client.create_collection(
      collection_name="test_collection",
      vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
   )

# operation_info = client.upsert(
#     collection_name="test_collection",
#     wait=True,
#     points=[
#         PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
#         PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
#         PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
#         PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
#         PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
#         PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
#     ],
# )

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(
            id=k,
            vector=v["embeddings"],
            payload={"query": v["queries"]},
        )
        for k, v in df_qrs.iterrows()
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [55]:
serarch_term = "dns"
limit = 4

query = generate_embeddings(serarch_term, model=model)
print(query)

# Which of our stored vectors are most similar to the query vector?
search_result = client.query_points(
    collection_name="test_collection",
    query=query,
    with_payload=True,
    limit=limit,
    score_threshold=0.5
).points

for i in search_result:
    print(i.payload)

[4.4576005935668945, -4.623528957366943, 5.292074680328369, -6.814428806304932, 2.09373140335083, 5.148980140686035, -3.5819036960601807, -2.0649256706237793, 7.729393482208252, -1.9043927192687988, 4.876160621643066, 5.461594581604004, 8.049408912658691, -6.559035778045654, 5.899397373199463, -0.4055189788341522, -1.5817443132400513, 1.7045578956604004, -4.775513648986816, -11.768861770629883, -3.2653324604034424, -2.3172316551208496, -1.5378131866455078, -3.2202305793762207, 1.2094835042953491, 2.921499252319336, 0.20934335887432098, -0.3293788433074951, -0.11273419857025146, -1.9783130884170532, -0.3122563064098358, 4.3912482261657715, 5.9114990234375, 6.9449238777160645, 2.147543430328369, 4.82988166809082, 2.0235910415649414, 7.903512954711914, -4.429738998413086, -0.8897181749343872, 7.834965705871582, -0.0350516140460968, 6.137450218200684, 5.689115524291992, 3.7314486503601074, -1.3162091970443726, -2.005178928375244, -7.762248992919922, -1.3171557188034058, 6.716493129730225, 

In [53]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=query,
    query_filter=Filter(
        must=[FieldCondition(key="query", match=MatchValue(value="checkdomain"))]
    ),
    with_payload=True,
    limit=limit,
).points

print(search_result)

[]
